# Import Libraries

In [33]:
import pandas as pd
import requests

import sys
sys.path.append('/home/adedapo/personal_project/daps05ayoade/DeepTrade')

from secrecy import api_key

# Get Stock Data

In [34]:
# Set the stock symbol (in this case, 'AAPL' for Apple Inc.)
ticker = 'IBM'

# Construct the API URL using the stock symbol and API token
api_url = f'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY_ADJUSTED&symbol={ticker}&apikey={api_key}'

# Send a GET request to the API and parse the JSON response
data = requests.get(api_url).json()

In [35]:
df = pd.DataFrame(data['Weekly Adjusted Time Series']).T

In [36]:
df.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount
2023-10-27,136.6300,144.7000,135.8700,142.5200,142.5200,30227448,0.0000
2023-10-20,139.2800,140.6200,136.3100,137.1600,137.1600,21044049,0.0000
2023-10-13,142.3000,143.4150,138.2700,138.4600,138.4600,16386334,0.0000
2023-10-06,140.0400,142.9400,139.8600,142.0300,142.0300,15932918,0.0000
2023-09-29,146.5700,147.4300,139.6100,140.3000,140.3000,23445425,0.0000


In [37]:
# Define new column names
column_names = ['open','high','low','close','adj_close','volume','dividend_amount']

# Assign new column names
df.columns = column_names

# Convert all column names to lower case
df.columns = df.columns.str.lower()

In [38]:
df.head()

,open,high,low,close,adj_close,volume,dividend_amount
2023-10-27,136.6300,144.7000,135.8700,142.5200,142.5200,30227448,0.0000
2023-10-20,139.2800,140.6200,136.3100,137.1600,137.1600,21044049,0.0000
2023-10-13,142.3000,143.4150,138.2700,138.4600,138.4600,16386334,0.0000
2023-10-06,140.0400,142.9400,139.8600,142.0300,142.0300,15932918,0.0000
2023-09-29,146.5700,147.4300,139.6100,140.3000,140.3000,23445425,0.0000


# Technical Data

## Exponential Moving Average (EMA)

- **The EMA gives more weight to the most recent prices**, and as such, it reacts more quickly to price changes than the SMA.
- It is calculated using the following formula:
  - **EMA<sub>t</sub> = (P<sub>t</sub> - EMA<sub>t-1</sub>) × (2 / (N + 1)) + EMA<sub>t-1</sub>**
- Where:
  - **P<sub>t</sub>**: is the price at time t
  - **EMA<sub>t-1</sub>**: is the EMA value at time t-1
  - **N**: is the span of the moving average.

In [39]:
ema_url = f'https://www.alphavantage.co/query?function=EMA&symbol={ticker}&interval=daily&time_period=10&series_type=close&apikey={api_key}'

ema = requests.get(ema_url).json()

df['ema'] = pd.DataFrame(ema['Technical Analysis: EMA']).T

In [40]:
df.head(2)

,open,high,low,close,adj_close,volume,dividend_amount,ema
2023-10-27,136.6300,144.7000,135.8700,142.5200,142.5200,30227448,0.0000,140.0438
2023-10-20,139.2800,140.6200,136.3100,137.1600,137.1600,21044049,0.0000,139.7183


## Moving Average Convergence Divergence (MACD)

The Moving Average Convergence Divergence (MACD) is a popular technical indicator used in financial analysis, particularly for stocks, to identify potential buy or sell signals. It's used to detect changes in the strength, direction, momentum, and duration of a trend in a stock's price.

### Construction:

1. **MACD Line**: This is the difference between two exponential moving averages (EMAs) of a stock’s price. Typically, the 12-day EMA minus the 26-day EMA.
   
   $$
   \text{MACD Line} = \text{12-day EMA} - \text{26-day EMA}
   $$

2. **Signal Line**: This is the 9-day EMA of the MACD line.

3. **Histogram**: This is the difference between the MACD line and the Signal line.

   $$
   \text{Histogram} = \text{MACD Line} - \text{Signal Line}
   $$

### Interpretation:

1. **Crossovers**:
    - **Bullish Crossover**: When the MACD line crosses above the Signal line, it's considered a bullish sign, suggesting it might be a good time to buy.
    - **Bearish Crossover**: When the MACD line crosses below the Signal line, it's considered a bearish sign, suggesting it might be a good time to sell.

2. **Divergence**:
   - If the price of a stock is making a new high, but the MACD is failing to surpass its previous high, this can be an indication of a potential price reversal to the downside (bearish divergence).
   - Conversely, if the stock price is making a new low, but the MACD is not reaching its previous lows, this can indicate a potential upward reversal (bullish divergence).

3. **Histogram**:
   - When the histogram is positive (above the zero line), it indicates the MACD is above the signal line, which can be a bullish sign.
   - When the histogram is negative (below the zero line), the MACD is below the signal line, indicating potential bearishness.

4. **Overextended MACD Values**:
   - If the MACD line moves too far away from the Signal line, it can be an indication that the stock is overbought or oversold and may soon return to normal levels.

In [41]:
macd_url = f'https://www.alphavantage.co/query?function=MACD&symbol={ticker}&interval=daily&time_period=10&series_type=close&apikey={api_key}'

macd = requests.get(macd_url).json()

macd_df = pd.DataFrame(macd['Technical Analysis: MACD']).T

In [42]:
df['macd'] = macd_df['MACD']
df['macd_signal'] = macd_df['MACD_Signal']
df['macd_hist'] = macd_df['MACD_Hist']

In [43]:
df.head(2)

,open,high,low,close,adj_close,volume,dividend_amount,ema,macd,macd_signal,macd_hist
2023-10-27,136.6300,144.7000,135.8700,142.5200,142.5200,30227448,0.0000,140.0438,-0.9347,-1.3147,0.3800
2023-10-20,139.2800,140.6200,136.3100,137.1600,137.1600,21044049,0.0000,139.7183,-1.5491,-1.1597,-0.3894


## Bollinger Bands (BBANDS)

Bollinger Bands is a technical analysis tool developed by John Bollinger in the 1980s. It is designed to provide a relative definition of high and low prices of a market instrument (like a stock) and to identify periods of high or low volatility. Bollinger Bands consist of three bands:

- **Middle Band**: A simple moving average (SMA).
- **Upper Band**: Calculated as the simple moving average plus a specified number of standard deviations (typically two).
- **Lower Band**: Calculated as the simple moving average minus a specified number of standard deviations (typically two).

The formula for the bands is as follows:

**Middle Band (MB)**: 
$$ MB = SMA(N) $$

Where \( SMA(N) \) is the simple moving average over \( (N) \) periods.

**Upper Band (UB)**: 
$$ UB = SMA(N) + (K \times \sigma(N)) $$

Where \( \sigma(N) \) is the standard deviation of the price over \( (N) \) periods, and \( (K) \) is a multiplier which is usually set to 2.

**Lower Band (LB)**: 
$$ LB = SMA(N) - (K \times \sigma(N)) $$

In [44]:
bbands_url = f'https://www.alphavantage.co/query?function=BBANDS&symbol={ticker}&interval=daily&time_period=10&series_type=close&apikey={api_key}'

bbands = requests.get(bbands_url).json()

bbands_df = pd.DataFrame(bbands['Technical Analysis: BBANDS']).T

In [45]:
df['upper_band'] = bbands_df['Real Upper Band']
df['middle_band'] = bbands_df['Real Middle Band']
df['lower_band'] = bbands_df['Real Lower Band']

In [46]:
df.head(2)

,open,high,low,close,adj_close,volume,dividend_amount,ema,macd,macd_signal,macd_hist,upper_band,middle_band,lower_band
2023-10-27,136.6300,144.7000,135.8700,142.5200,142.5200,30227448,0.0000,140.0438,-0.9347,-1.3147,0.3800,143.8534,139.2200,134.5866
2023-10-20,139.2800,140.6200,136.3100,137.1600,137.1600,21044049,0.0000,139.7183,-1.5491,-1.1597,-0.3894,143.9869,140.1910,136.3951


## Relative Strength Index (RSI)

The Relative Strength Index (RSI) is a momentum oscillator that measures the speed and change of price movements. It was developed by J. Welles Wilder and introduced in his 1978 book, "New Concepts in Technical Trading Systems." RSI is used to identify overbought or oversold conditions in a traded security.

### Calculation:

RSI is calculated using the following formula:

$$ \text{RSI} = 100 - \frac{100}{1 + RS} $$

Where:

$$ RS = \frac{\text{Average Gain over n periods}}{\text{Average Loss over n periods}} $$

1. Compute the average gain and average loss over a specified period, typically 14 days.
2. Calculate the relative strength (RS), which is the ratio of average gain to average loss.
3. Calculate the RSI using the formula given above.

### Interpretation:

- The RSI oscillates between 0 and 100.
- Traditionally, an RSI above 70 indicates that a security is overbought, suggesting it might be overvalued and is a potential candidate for a price pullback or a reversal.
- Conversely, an RSI below 30 indicates that a security is oversold, suggesting it might be undervalued and is a potential candidate for a price rally or a reversal.
- The standard setting for RSI is a 14-day period, but traders sometimes use different periods based on their strategies and the asset they're trading.

In [47]:
rsi_url = f'https://www.alphavantage.co/query?function=RSI&symbol={ticker}&interval=daily&time_period=10&series_type=close&apikey={api_key}'

rsi = requests.get(rsi_url).json()

df['rsi'] = pd.DataFrame(rsi['Technical Analysis: RSI']).T

In [48]:
df.head(2)

,open,high,low,close,adj_close,volume,dividend_amount,ema,macd,macd_signal,macd_hist,upper_band,middle_band,lower_band,rsi
2023-10-27,136.6300,144.7000,135.8700,142.5200,142.5200,30227448,0.0000,140.0438,-0.9347,-1.3147,0.3800,143.8534,139.2200,134.5866,56.9752
2023-10-20,139.2800,140.6200,136.3100,137.1600,137.1600,21044049,0.0000,139.7183,-1.5491,-1.1597,-0.3894,143.9869,140.1910,136.3951,30.2311


## Stochastic Oscillator (STOCH)

The Stochastic Oscillator is a momentum indicator that measures the position of a stock's latest closing price relative to its high and low range over a specific time period. It provides insights into potential overbought or oversold conditions in the stock, helping traders to identify potential trend reversals.

The formula for the Stochastic Oscillator is:

$$ \%K = \left( \frac{\text{Latest Close} - \text{Lowest Low}}{\text{Highest High} - \text{Lowest Low}} \right) \times 100 $$

Where:
- **Latest Close** is the most recent closing price.
- **Lowest Low** is the lowest price of the stock over the specified period.
- **Highest High** is the highest price of the stock over the specified period.

The Stochastic Oscillator comprises two lines:
1. **%K** – The main line, calculated from the above formula.
2. **%D** – A moving average of the %K value, usually a 3-period simple moving average.

### Interpretation:

- **Overbought & Oversold Levels**: Typically, the Stochastic Oscillator ranges from 0 to 100. A value above 80 is usually considered "overbought," suggesting that a price reversal or correction might be imminent. Conversely, a value below 20 is considered "oversold," indicating potential for a price rise.
  
- **Signal Line Crossovers**: When the %K line crosses above the %D line, it's considered a bullish signal (potential buy), especially if this crossover happens below the 20 level. When the %K line crosses below the %D line, it's seen as a bearish signal (potential sell), especially if this happens above the 80 level.

- **Divergence**: If the stock's price forms a new high or low that isn't confirmed by the Stochastic Oscillator, it may indicate a potential trend reversal. For example, if a stock forms a new high, but the Stochastic Oscillator doesn't, it's a bearish divergence.

In [49]:
stoch_url = f'https://www.alphavantage.co/query?function=STOCH&symbol={ticker}&interval=daily&time_period=10&series_type=close&apikey={api_key}'

stoch = requests.get(stoch_url).json()

stoch_df = pd.DataFrame(stoch['Technical Analysis: STOCH']).T

In [50]:
df['slowk'] = stoch_df['SlowK']
df['slowd'] = stoch_df['SlowD']

In [51]:
df.head(2)

,open,high,low,close,adj_close,volume,dividend_amount,ema,macd,macd_signal,macd_hist,upper_band,middle_band,lower_band,rsi,slowk,slowd
2023-10-27,136.6300,144.7000,135.8700,142.5200,142.5200,30227448,0.0000,140.0438,-0.9347,-1.3147,0.3800,143.8534,139.2200,134.5866,56.9752,67.3098,50.8582
2023-10-20,139.2800,140.6200,136.3100,137.1600,137.1600,21044049,0.0000,139.7183,-1.5491,-1.1597,-0.3894,143.9869,140.1910,136.3951,30.2311,38.1506,44.0913


## Average True Range (ATR)

The Average True Range (ATR) is a technical indicator that measures the volatility of a stock or any other market instrument. It was introduced by J. Welles Wilder in his 1978 book "New Concepts in Technical Trading Systems." ATR does not provide an indication of price direction but instead quantifies the degree of price volatility.

### Calculation:

1. **True Range Calculation**:

   First, you need to calculate the True Range (TR) for each period. The TR for any period is calculated as the greatest of the following:

   - Current high minus the current low.
   - Absolute value of the current high minus the previous close.
   - Absolute value of the current low minus the previous close.

   Mathematically:

   $$
   \text{TR} = \max[(\text{High} - \text{Low}), |\text{High} - \text{Previous Close}|, |\text{Low} - \text{Previous Close}|]
   $$

2. **Average True Range Calculation**:

   The ATR is typically calculated using a 14-day moving average of the TR values, though the period can be adjusted based on the user's preferences. Wilder originally used a smoothed moving average, but many traders now use an exponential moving average.

   For the first period's ATR value:

   $$
   \text{ATR}_1 = \frac{1}{n} \sum_{i=1}^{n} \text{TR}_i
   $$

   Where:
   - \( n \) is the number of periods, typically 14.
   - \( \text{TR}_i \) is the True Range of the ith period.

   For subsequent days:

   $$
   \text{ATR}_{t} = \frac{\text{Previous ATR} \times (n-1) + \text{TR}_{t}}{n}
   $$

   This formula gives the new ATR a weight relative to the previous ATR, with the latest day's TR receiving the most weight.

### Interpretation and Use:

- The ATR value rises when price movements (either up or down) are large, and it diminishes when price changes are minimal, making it a reflection of volatility.
  
- ATR can be used to place stop-loss orders. For instance, a trader might set a stop-loss at a multiple of the ATR below their entry price for a long position.

- It can be used as a filter for trading systems, where trades might only be considered if the ATR is above a certain level, indicating sufficient volatility for potential profitable price movements.

- ATR does not indicate price direction and only provides an estimate of volatility. It's often used in conjunction with other indicators to develop trading strategies.

In [52]:
atr_url = f'https://www.alphavantage.co/query?function=ATR&symbol={ticker}&interval=daily&time_period=10&series_type=close&apikey={api_key}'

atr = requests.get(atr_url).json()

df['atr'] = pd.DataFrame(atr['Technical Analysis: ATR']).T

In [53]:
df.head(2)

,open,high,low,close,adj_close,volume,dividend_amount,ema,macd,macd_signal,macd_hist,upper_band,middle_band,lower_band,rsi,slowk,slowd,atr
2023-10-27,136.6300,144.7000,135.8700,142.5200,142.5200,30227448,0.0000,140.0438,-0.9347,-1.3147,0.3800,143.8534,139.2200,134.5866,56.9752,67.3098,50.8582,2.6151
2023-10-20,139.2800,140.6200,136.3100,137.1600,137.1600,21044049,0.0000,139.7183,-1.5491,-1.1597,-0.3894,143.9869,140.1910,136.3951,30.2311,38.1506,44.0913,2.0695


# Derived Features

## Daily Returns

Daily returns measure how much the value of an investment has gone up or down compared to the previous day. They are usually expressed as a percentage.

### How is it Calculated?

For a stock, if the closing price today is higher than yesterday, the daily return is positive, showing that the value of your investment has increased. Conversely, if the closing price today is lower than yesterday, the daily return is negative, indicating a decrease in your investment value.

### Formula:

If \( P_t \) is today's closing price and \( P_{t-1} \) is yesterday's closing price, the daily return \( R_t \) can be calculated as:

$$
R_t = \left( \frac{P_t - P_{t-1}}{P_{t-1}} \right) \times 100 \%
$$

### Why is it Useful?

Daily returns provide a straightforward way to monitor the performance of an investment on a day-to-day basis. Investors and analysts use daily returns to understand the volatility or risk of an investment and to compare the performance of different investments.

In [54]:
# Set the index to datetime
df.index = pd.to_datetime(df.index)

# Sort Index in ascending order
df = df.sort_index(ascending=True)

# Convert all columns to float
df = df.astype(float)

df['returns'] = df['adj_close'].pct_change()

In [58]:
df.iloc[4:7]

,open,high,low,close,adj_close,volume,dividend_amount,ema,macd,macd_signal,macd_hist,upper_band,middle_band,lower_band,rsi,slowk,slowd,atr,returns
1999-12-10,113.00,122.12,107.56,109.00,57.9265,58626000.0,0.0,58.7157,NaN,NaN,NaN,64.4494,58.5153,52.5812,53.6128,44.2517,67.3747,2.4450,-0.025654
1999-12-17,108.12,112.75,104.50,110.00,58.4579,38810100.0,0.0,58.2416,2.2876,3.1375,-0.8499,63.3726,59.4363,55.5000,55.2716,45.9012,30.8539,2.3108,0.009174
1999-12-23,109.06,110.44,107.75,108.62,57.7245,18144100.0,0.0,58.0350,1.6304,2.3688,-0.7384,58.8830,57.9307,56.9785,50.9433,51.4425,64.2224,1.9492,-0.012546


## Rolling standard deviation of returns

The rolling standard deviation of returns is a measure that shows us how much the return of a stock varies or 'moves around' over a certain period. Think of 'returns' as the percentage change in the stock's price from one day to the next.

To understand this, imagine we look at a stock’s daily price changes over 20 days. On some days, the price might go up a lot, and on others, it might fall or not change much. The rolling standard deviation tells us, on average, how much these daily changes in price differ from their average change over these 20 days.

$$
\text{Return}_t = \frac{\text{Price}_t - \text{Price}_{t-1}}{\text{Price}_{t-1}}
$$

A higher rolling standard deviation means the stock’s price is fluctuating more during that period – it's more volatile. If the stock's price moves up and down only a little each day, the rolling standard deviation will be lower, indicating that the stock is less volatile.

This measure helps investors understand the risk associated with the stock's price changes. Generally, a more volatile stock is considered riskier, as its price can change dramatically in a short period, making its future price harder to predict. Conversely, a stock with lower volatility tends to have more predictable price movements, which some investors might prefer, especially if they're more risk-averse.

**Tips for Readers:**
1. **Time Window**: The choice of the window size (e.g., 10 days) can affect the rolling standard deviation. A smaller window will be more sensitive to recent price changes, while a larger window smooths out short-term fluctuations, offering a view of longer-term volatility.
2. **Use in Investment Decisions**: Investors use the rolling standard deviation of returns to assess the risk level of a stock and to compare the risk profiles of different stocks. It is also used in portfolio construction and risk management.
3. **Not a Predictor of Future Movements**: It's important to note that while this statistic helps in understanding historical volatility, it doesn't necessarily predict future price movements or volatility.

In [59]:
df['rolling_std'] = df['returns'].rolling(10).std()

In [63]:
df.iloc[9:12]

,open,high,low,close,adj_close,volume,dividend_amount,ema,macd,macd_signal,macd_hist,upper_band,middle_band,lower_band,rsi,slowk,slowd,atr,returns,rolling_std
2000-01-14,117.25,123.31,115.37,119.62,63.5703,42663900.0,0.0,61.9210,1.8824,1.6822,0.2002,64.6855,61.9617,59.2378,65.7268,66.2018,73.5742,2.5228,0.053921,NaN
2000-01-21,119.69,124.75,112.69,121.50,64.5694,41930500.0,0.0,62.7513,1.8077,1.7319,0.0758,65.1674,62.9018,60.6361,64.5487,63.1658,49.0716,2.7276,0.015716,0.036908
2000-01-28,121.87,122.87,110.06,111.56,59.2869,33325200.0,0.0,61.8767,0.9035,1.4748,-0.5714,65.9450,62.5925,59.2399,38.5602,12.2671,27.7944,2.7590,-0.081811,0.043803


# Set Target Column

In [26]:
# Create target column
df['target'] = df['adj_close'].shift(-1)

# Drop NA values
df.dropna(inplace=True)

In [27]:
df.head(2)

,open,high,low,close,adj_close,volume,dividend_amount,ema,macd,macd_signal,...,upper_band,middle_band,lower_band,rsi,slowk,slowd,atr,returns,rolling_std,target
2000-01-21,119.69,124.75,112.69,121.50,64.5694,41930500.0,0.0,62.7513,1.8077,1.7319,...,65.1674,62.9018,60.6361,64.5487,63.1658,49.0716,2.7276,0.015716,0.036908,59.2869
2000-01-28,121.87,122.87,110.06,111.56,59.2869,33325200.0,0.0,61.8767,0.9035,1.4748,...,65.9450,62.5925,59.2399,38.5602,12.2671,27.7944,2.7590,-0.081811,0.043803,61.4446


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1240 entries, 2000-01-21 to 2023-10-20
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   open             1240 non-null   float64
 1   high             1240 non-null   float64
 2   low              1240 non-null   float64
 3   close            1240 non-null   float64
 4   adj_close        1240 non-null   float64
 5   volume           1240 non-null   float64
 6   dividend_amount  1240 non-null   float64
 7   ema              1240 non-null   float64
 8   macd             1240 non-null   float64
 9   macd_signal      1240 non-null   float64
 10  macd_hist        1240 non-null   float64
 11  upper_band       1240 non-null   float64
 12  middle_band      1240 non-null   float64
 13  lower_band       1240 non-null   float64
 14  rsi              1240 non-null   float64
 15  slowk            1240 non-null   float64
 16  slowd            1240 non-null   float64
 

# Modualised

In [64]:
def get_data_from_url(url: str):
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception if the request was unsuccessful
    return response.json()

def get_technical_data(symbol: str, feature: str, api_key: str):
    url = f'https://www.alphavantage.co/query?function={feature}&symbol={symbol}&interval=daily&time_period=10&series_type=close&apikey={api_key}'
    data = get_data_from_url(url)
    key = 'Technical Analysis: ' + feature
    
    # Check if the response contains an error
    if "Error Message" in data:
        raise ValueError(f"Error retrieving {feature} for {symbol}: {data['Error Message']}")
    if key not in data:
        raise ValueError(f"Unexpected API response structure when fetching {feature} for {symbol}")
    
    if feature == 'BBANDS':
        df = pd.DataFrame(data[key]).T
        df = df.rename(columns={
            'Real Upper Band': 'upper_band',
            'Real Middle Band': 'middle_band',
            'Real Lower Band': 'lower_band'
        })
        for col in ['upper_band', 'middle_band', 'lower_band']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
    elif feature == 'MACD':
        df = pd.DataFrame(data[key]).T
        df = df.rename(columns={
            'MACD': 'macd',
            'MACD_Signal': 'macd_signal',
            'MACD_Hist': 'macd_hist'
        })
        for col in ['macd', 'macd_signal', 'macd_hist']:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    elif feature == 'STOCH':
        df = pd.DataFrame(data[key]).T
        df = df.rename(columns={
            'SlowK': 'slowk',
            'SlowD': 'slowd'
        })
        for col in ['slowk', 'slowd']:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    else:
        df = pd.DataFrame(data[key]).T

    return df

def get_stock_data(symbol: str, api_key: str):
    api_url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={api_key}'

    # Obtain and process stock data
    data = get_data_from_url(api_url)
    df = pd.DataFrame(data['Time Series (Daily)']).T

    # Assign new column names
    column_names = ['open','high','low','close','adj_close','volume', 'dividend', 'split_coeff']
    df.columns = column_names

    # Convert columns to numeric
    for col in column_names:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Obtain technical data and merge
    features = ['EMA', 'MACD', 'BBANDS', 'RSI', 'STOCH', 'ATR']
    for feature in features:
        tech_df = get_technical_data(symbol, feature, api_key)
        df = df.join(tech_df, rsuffix=f'_{feature}')
        
    # Convert all column names to lower case
    df.columns = df.columns.str.lower()
    
    # Set the index to datetime
    df.index = pd.to_datetime(df.index)
    
    # Sort Index in ascending order
    df = df.sort_index(ascending=True)
    
    df['returns'] = df['adj_close'].pct_change()
    
    df['rolling_std'] = df['returns'].rolling(10).std()

    # Create target column
    df['target'] = df['adj_close'].shift(-1)
    
    # Drop NA values
    df.dropna(inplace=True)

    return df

In [65]:
df_2 = get_stock_data('AAPL', api_key)

In [66]:
df_2.head()

,open,high,low,close,adj_close,volume,dividend,split_coeff,ema,macd,...,upper_band,middle_band,lower_band,rsi,slowk,slowd,atr,returns,rolling_std,target
1999-12-17,100.87,102.00,98.50,100.00,0.757759,4419700,0.0,1.0,0.7652,0.0243,...,0.9044,0.7891,0.6737,50.9703,49.2644,27.6202,0.0438,0.017191,0.032402,0.742604
1999-12-20,99.56,99.62,96.62,98.00,0.742604,2535600,0.0,1.0,0.7611,0.0211,...,0.8764,0.7754,0.6744,47.4399,60.4803,44.3777,0.0420,-0.020000,0.031486,0.776703
1999-12-21,98.19,103.06,97.94,102.50,0.776703,2746400,0.0,1.0,0.7639,0.0211,...,0.8283,0.7638,0.6994,55.1978,78.9724,62.9057,0.0416,0.045918,0.036027,0.757304
1999-12-22,102.87,104.56,98.75,99.94,0.757304,2920300,0.0,1.0,0.7627,0.0193,...,0.8005,0.7561,0.7118,50.4874,71.6734,70.3754,0.0419,-0.024976,0.031432,0.784281
1999-12-23,101.81,104.25,101.06,103.50,0.784281,2049400,0.0,1.0,0.7666,0.0198,...,0.7947,0.7548,0.7150,56.2554,79.4111,76.6856,0.0410,0.035621,0.031752,0.752530


In [67]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6003 entries, 1999-12-17 to 2023-10-26
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   open         6003 non-null   float64
 1   high         6003 non-null   float64
 2   low          6003 non-null   float64
 3   close        6003 non-null   float64
 4   adj_close    6003 non-null   float64
 5   volume       6003 non-null   int64  
 6   dividend     6003 non-null   float64
 7   split_coeff  6003 non-null   float64
 8   ema          6003 non-null   object 
 9   macd         6003 non-null   float64
 10  macd_signal  6003 non-null   float64
 11  macd_hist    6003 non-null   float64
 12  upper_band   6003 non-null   float64
 13  middle_band  6003 non-null   float64
 14  lower_band   6003 non-null   float64
 15  rsi          6003 non-null   object 
 16  slowk        6003 non-null   float64
 17  slowd        6003 non-null   float64
 18  atr          6003 non-null   o